In [1]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, AutoConfig
import os
from datasets import load_from_disk
os.environ['TRANSFORMERS_CACHE'] = '../transformers_cache'



In [ ]:
hf_token = os.getenv("hf_token")
# model_name = "meta-llama/Llama-3.1-70B-Instruct"
# model_name = "meta-llama/Llama-3.1-8B-Instruct"

config = AutoConfig.from_pretrained(model_name, token=hf_token)
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token, config=config,cache_dir='../transformers_cache')
model = AutoModelForCausalLM.from_pretrained(model_name, token=hf_token, config=config,cache_dir='../transformers_cache')
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = 'left'
# model.resize_token_embeddings(len(tokenizer))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
model.to(device)

In [ ]:
import pandas as pd

In [ ]:
path = '../outputs/gsm8k/LLaMA1B/generated_outputs.json'
df = pd.read_json(path)

In [ ]:
df=df[df['score']==0]

In [ ]:
df.shape